In [ ]:
%%capture
import ROOT
import glob
import math
import sys
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.drawMultiTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *
from TPCQCVis.src.utility import *
import warnings
from copy import copy
warnings.filterwarnings('ignore')

In [ ]:
%jsroot on
display(HTML("<style>.container { width:95% !important; align-items: center;}</style>"))
display(HTML("<style>table {float:left;}</style>"))
ROOT.gErrorIgnoreLevel = ROOT.kFatal
#display(HTML('<style>{}</style>'.format(CSS)))
#ROOT.gStyle.SetPalette(57)

### Quality of life functions

In [ ]:
def checkIfExists(files, title):
    objectAvailable = False
    hist = None
    goodFiles = []
    if type(files) is list:
        file = files[0]
        try:
            hist = file.PIDQC.Get(title)
            if not hist : hist = file.TracksQC.Get(title)
            if not hist : hist = file.ClusterQC.Get(title)
            if not hist : hist = file.PID.Get(title)
            if not hist : hist = file.Tracks.Get(title)
        except:
            objectAvailable = False
        else:
            if hist:
                objectAvailable = True
            else:
                objectAvailable = False
        if not objectAvailable:
            print(title,"not found")
    else:
        for file in files:
            try:
                hist = file.PIDQC.Get(title)
                if not hist : hist = file.TracksQC.Get(title)
                if not hist : hist = file.ClusterQC.Get(title)
                if not hist : hist = file.PID.Get(title)
                if not hist : hist = file.Tracks.Get(title)
            except:
                objectAvailable = False
            else:
                if hist:
                    objectAvailable = True
                    goodFiles.append(file)
                else:
                    objectAvailable = False
            if not objectAvailable:
                print(title,"not found in", file)
        files = goodFiles
        if len(files):
            objectAvailable = True
    return objectAvailable

## Read files

In [ ]:
# Notebook variables
periodName = "LHC23zzk"
passName = "apass1"
runNumber = "544508"
path = "/cave/alice-tpc-qc/data/2023/"
# Read the Root Files
fileList = glob.glob(path+"/"+periodName+"/"+passName+"/"+"*_QC.root")
fileList = [file for file in fileList if file[-13] != "_"]
fileList.sort()
#fileList = fileList[13:]
runList = [fileList[i][-14:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))

In [ ]:
ROOT.gStyle.SetPalette(57)
ROOT.gStyle.SetPalette(55)
ROOT.gStyle.SetGridStyle(3)
ROOT.gStyle.SetGridWidth(1)
ROOT.gStyle.SetOptStat(0)
cols = list(ROOT.TColor.GetPalette())
def updateColors(histograms,palette):
    colors = []
    for i,hist in enumerate(histograms):
        if len(histograms)>1:
            color = palette[math.floor((i/(len(histograms)-1))*(len(palette)-1))]
        else:
            color = palette[0]
        #color = i
        colors.append(color)
        hist.SetLineColor(color)
        hist.SetMarkerColor(color)
    return colors

def updateRanges(histograms):
    maxRange = max([hist.GetMaximum() for hist in histograms])
    minRange = min([hist.GetMinimum() for hist in histograms])
    for i,hist in enumerate(histograms):
        if len(histograms)>1:
            hist.SetMaximum(maxRange)
            hist.SetMinimum(maxRange)

In [ ]:
%jsroot on
objectName="hEta"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",xAxisRange=[-1.1,1.1],grid=True)
    updateColors(hist,cols)
    pad1.SetGrid(1)
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DEtaPhi;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-1,1])
    pad1.cd()
    sectorBoundary = ROOT.TLine(0,hist[0].GetMinimum(),0,hist[0].GetMaximum())
    sectorBoundary.SetLineStyle(2)
    sectorNum = ROOT.TText(.5,.5,"test")
    sectorNum.SetTextAlign(22)
    sectorNum.SetTextColor(ROOT.kGray+1)
    sectorNum.SetTextFont(0)
    for j in range(19):
        sectorBoundary.DrawLine((j*2*math.pi)/18,-1,(j*2*math.pi)/18,1)
        if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,1.05,str(j))
    canvas.SetCanvasSize(1000,400)
    #hist[0].GetXaxis().SetRangeUser(-1,1)
    #hist[0].GetYaxis().SetRangeUser(60,180)
    canvas.Draw()

## Build median histogram

In [ ]:
# def getMedianHistogram(hists):
#     from statistics import median
#     if len(hists) == 1:
#         return hists[0]
#     else:
#         nBins = hist[0].GetNbinsX()
#         xMin = hist[0].GetXaxis().GetXmin()
#         xMax = hist[0].GetXaxis().GetXmax()
#         medianHist = ROOT.TH1F("median_"+hist[0].GetName(),"Median "+hist[0].GetTitle(),nBins,xMin,xMax)
#         for xBin in range(nBins):
#             vals = [h.GetBinContent(xBin) for h in hist]
#             medianHist.SetBinContent(xBin,median(vals))
#         return medianHist

def getMedianHistogram(hists):
    from statistics import median

    if len(hists) == 0:
        raise ValueError("Histogram list is empty")

    # Determine if histograms are 1D or 2D
    is2D = isinstance(hists[0], ROOT.TH2)

    if len(hists) == 1:
        return hists[0]

    if is2D:
        # 2D histogram case
        nBinsX = hists[0].GetNbinsX()
        nBinsY = hists[0].GetNbinsY()
        xMin = hists[0].GetXaxis().GetXmin()
        xMax = hists[0].GetXaxis().GetXmax()
        yMin = hists[0].GetYaxis().GetXmin()
        yMax = hists[0].GetYaxis().GetXmax()
        medianHist = ROOT.TH2F("median_" + hists[0].GetName(), "Median " + hists[0].GetTitle(), nBinsX, xMin, xMax, nBinsY, yMin, yMax)
        for xBin in range(1, nBinsX + 1):
            for yBin in range(1, nBinsY + 1):
                vals = [h.GetBinContent(xBin, yBin) for h in hists]
                medianHist.SetBinContent(xBin, yBin, median(vals))
    else:
        # 1D histogram case
        nBins = hists[0].GetNbinsX()
        xMin = hists[0].GetXaxis().GetXmin()
        xMax = hists[0].GetXaxis().GetXmax()
        medianHist = ROOT.TH1F("median_" + hists[0].GetName(), "Median " + hists[0].GetTitle(), nBins, xMin, xMax)
        for xBin in range(1, nBins + 1):
            vals = [h.GetBinContent(xBin) for h in hists]
            medianHist.SetBinContent(xBin, median(vals))

    return medianHist


In [ ]:
%jsroot on
objectName="hEta"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,legend=True,legendNames=runList,pads=False,
                                               drawOption="SAME L",xAxisRange=[-1.1,1.1],grid=True)
    updateColors(hist,cols)
    pad1.SetGrid(1)
    pad1.cd()
    medianHist = getMedianHistogram(hist)
    medianHist.Draw("SAME *")
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DEtaPhi;1"
if checkIfExists(rootDataFile,objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-1,1])
    updateColors(hist,cols)
    pad1.SetGrid(1)
    pad1.cd()
    medianHist = getMedianHistogram(hist)
    medianHist.Draw("SAME *")
    canvas.SetCanvasSize(700,400)
    canvas.Draw()

In [ ]:
def compareToMedian(hists, medianHist, normalize = True, distance = 0.1):
    if normalize:
        medianHist.Scale(medianHist.Integral())
    is2D = isinstance(medianHist, ROOT.TH2) # Determine if histograms are 1D or 2D
    if is2D:
        # Handle 2D histograms
        nBinsX = medianHist.GetNbinsX()
        nBinsY = medianHist.GetNbinsY()
        nBins = nBinsX * nBinsY
        for hist in hists:
            for xBin in range(1, nBinsX + 1):
                for yBin in range(1, nBinsY + 1):
                    medianValue = medianHist.GetBinContent(xBin, yBin)
                    currentValue = hist.GetBinContent(xBin, yBin)
                    if medianValue != 0:  # To avoid division by zero
                        difference = abs(currentValue - medianValue) / medianValue
                        if difference > distance:  # More than 10% difference
                            xAxisValue = medianHist.GetXaxis().GetBinCenter(xBin)
                            yAxisValue = medianHist.GetYaxis().GetBinCenter(yBin)
                            print(f"Run: {hist.GetName()}, X-axis Value: {xAxisValue}, Y-axis Value: {yAxisValue}, Difference: {difference*100}%")
    else:
        # Handle 1D histograms
        nBinsX = medianHist.GetNbinsX()
        for hist in hists:
            for xBin in range(1, nBinsX + 1):
                medianValue = medianHist.GetBinContent(xBin)
                currentValue = hist.GetBinContent(xBin)
                if medianValue != 0:  # To avoid division by zero
                    difference = abs(currentValue - medianValue) / medianValue
                    if difference > distance:  # More than 10% difference
                        xAxisValue = medianHist.GetXaxis().GetBinCenter(xBin)
                        print(f"Run: {hist.GetName()}, X-axis Value: {xAxisValue}, Difference: {difference*100}%")


def checkAndPrintSignificantDifferences(hists, medianHist, normalize = True, distance = 0.1):
    if normalize:
        
    is2D = isinstance(medianHist, ROOT.TH2) # Determine if histograms are 1D or 2D
    if is2D:
        # Handle 2D histograms
        nBinsX = medianHist.GetNbinsX()
        nBinsY = medianHist.GetNbinsY()
        nBins = nBinsX * nBinsY
        for hist in hists:
            for xBin in range(1, nBinsX + 1):
                for yBin in range(1, nBinsY + 1):
                    medianValue = medianHist.GetBinContent(xBin, yBin)
                    currentValue = hist.GetBinContent(xBin, yBin)
                    if medianValue != 0:  # To avoid division by zero
                        difference = abs(currentValue - medianValue) / medianValue
                        if difference > distance:  # More than 10% difference
                            xAxisValue = medianHist.GetXaxis().GetBinCenter(xBin)
                            yAxisValue = medianHist.GetYaxis().GetBinCenter(yBin)
                            print(f"Run: {hist.GetName()}, X-axis Value: {xAxisValue}, Y-axis Value: {yAxisValue}, Difference: {difference*100}%")
    else:
        # Handle 1D histograms
        nBinsX = medianHist.GetNbinsX()
        for hist in hists:
            for xBin in range(1, nBinsX + 1):
                medianValue = medianHist.GetBinContent(xBin)
                currentValue = hist.GetBinContent(xBin)
                if medianValue != 0:  # To avoid division by zero
                    difference = abs(currentValue - medianValue) / medianValue
                    if difference > distance:  # More than 10% difference
                        xAxisValue = medianHist.GetXaxis().GetBinCenter(xBin)
                        print(f"Run: {hist.GetName()}, X-axis Value: {xAxisValue}, Difference: {difference*100}%")


# Main part of the script
%jsroot on
objectName = "h2DEtaPhi;1"
if checkIfExists(rootDataFile, objectName):
    [hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,drawOption="COLZ",yAxisRange = [-1,1])
    updateColors(hist, cols)
    pad1.SetGrid(1)
    pad1.cd()
    medianHist = getMedianHistogram(hist)
    medianHist.Draw("SAME *")
    checkAndPrintSignificantDifferences(hist, medianHist)
    canvas.SetCanvasSize(700, 400)
    canvas.Draw()
    

In [ ]:

# Main part of the script
%jsroot on
objectName = "hEta"
if checkIfExists(rootDataFile, objectName):
    [hist, legend, canvas, pad1] = drawHistograms(objectName, rootDataFile, normalize=True, legend=True, legendNames=runList, pads=False, drawOption="SAME L", xAxisRange=[-1.1, 1.1], grid=True)
    updateColors(hist, cols)
    pad1.SetGrid(1)
    pad1.cd()
    medianHist = getMedianHistogram(hist)
    medianHist.Draw("SAME *")
    checkAndPrintSignificantDifferences(hist, medianHist)
    canvas.SetCanvasSize(700, 400)
    canvas.Draw()

In [ ]:
hist[0].GetNbinsY()